In [ ]:
from sdk import datalake
from sdk.datalake_publishing import DPCPublishing
import pandas as pd
from io import BytesIO

In [ ]:
# Get a list of new files
files_updated = datalake.batch_files_recent('length_extract','length_transform')
    

In [ ]:
            # If there are any new files, process them in turn
if files_updated:  

#    for length_file in files_updated:     
#    x=0    
    for l_file in datalake.list_batch_files('recorded_length_extract'): 


    #    print(l_file + str(2))
        output_file = length_file.replace('recorded_length','aggregated_length')
        
         # exclude the file names starting with 'recorded_m'
        if l_file.startswith('recorded_m'): 
            continue

        # load the lengths data in inches from the Data Lake, and import the parquet file into a dataframe

        length_content_in = datalake.load_from_extract('recorded_length_in_extract', length_in_file)  
    #    print(l_file + str(3))
        df_in = pd.read_parquet(BytesIO(length_content_in))
    #    print(l_file + str(4))

            # Total by Item Name and by Year, and reset the index to flatten the column headings
        df_in = df_in.groupby(['Item Name','Year','Quantity','Measure_Type']).sum().reset_index()
    #    print(l_file + str(5))

            # load the length data in cm from the Data Lake, and import the parquet file into a dataframe
        length_content_cm = datalake.load_from_extract('recorded_length_cm_extract', length_cm_file)  
    #    print(l_file + str(6))
        df_cm = pd.read_parquet(BytesIO(length_content_cm))
    #    print(l_file + str(7))

        # if the rows are not empty
        if not df_cm.empty:
    #        print(l_file + str(8))
            df_cm = df_cm.groupby(['Item Name', 'Year', 'Quantity', 'Measure_Type']).sum().reset_index()
    #        print(l_file + str(9))
            df_cm['Length_in'] = df_cm['Length'].apply(lambda x: x/2.54 if x>0 else 0)
    #        print(l_file + str(10))
            df_cm_converted = df_gas[['Item Name','Year','Quantity','Measure_Type','Length_in']].copy()
    #        print(l_file + str(11))
            df_cm_converted['Unit'] = 'inches'
    #        print(l_file + str(12))
            df_cm_converted.rename(columns={'Length_in':'Length'}, inplace=True)

            # total by Item Name and by Year, and reset the index to flatten the column headings

               df_cm_converted = df_cm.groupby(['Item Name','Year','Quantity','Measure_Type']).sum().reset_index()

            # Create a combined dataframe with both inches and cm:
            df_total = pd.concat([df_in, df_cm_converted], ignore_index=True)
          
            df_total = df_total.groupby(['Item Name','Year','Quantity','Measure_Type']).sum().reset_index()
    

        # combine all three datframes
            df_final = pd.concat([df_cm, df_in, df_total], ignore_index=True)
            df_final.insert(0, "Metric", 'Reported Sales')
            df_final = df_final.groupby(['Item Name','Metric','Year','Quantity','Fabric_Type']).sum().reset_index()
            df_final = df_final[['Item Name','Metric', 'Fabric_Type','Year','Quantity','Length']].sort_values(by=['Item Name', 'Year', 'Fabric_Type'])

            df_final= df_final.loc[df_final['Year'] != 0]

#           print(df_final)

            cnt=df_final['Item Name'].nunique()
            x+=cnt       
    
    #        print(df_in['Company Name'].nunique())
    
                  #Save new file
            datalake.save_transform_parquet('length_aggregated_transform', df_final, output_file)

    print ('\n' f'COUNT: {x}')  # how many items get processed 
#  
#    print(cm_file + str(13))

#    print(cm_file + str(14)) 
